In [ ]:
from ps_model import load_model
from test_personsearch_prw_sys import get_prw_dataset_info, get_prw_data, search_performance_prw
import torch
dataset = 'PRW'
info = {
    'PRW':{
        "config":"/home/linhuadong/DNFNet/jobs/prw_SC_STC_0p001/work_dirs/prw/prw.py",
        "checkpoint":"/home/linhuadong/DNFNet/jobs/prw_SC_STC_0p001/work_dirs/prw/latest.pth"
    },
    'CUHK':{
        
    }
}

In [ ]:
PRW_Dataset, pname_to_attribute, gt_roidb, name_to_det_feat = get_prw_dataset_info()

In [ ]:
model = load_model(info[dataset], 6)

In [ ]:
idx = 17
data = get_prw_data(PRW_Dataset, pname_to_attribute, idx)
with torch.no_grad():
    result = model(return_loss=False, rescale=True, **data)
results = search_performance_prw(result, data, pname_to_attribute, name_to_det_feat, gt_roidb)

In [ ]:
def visualize(image_root, probe_img, probe_roi, color):
    file_root = os.path.join(image_root, probe_img)
    img = Image.open(file_root)
    plt.figure(figsize=(8, 8))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    
    ax = plt.gca()
    down_left_x, down_left_y = probe_roi[0], probe_roi[1]
    width = probe_roi[2] - probe_roi[0]
    height = probe_roi[3] - probe_roi[1]
    rect = plt.Rectangle((down_left_x, down_left_y), width, height, fill=False, edgecolor=color, linewidth=1)
    ax.add_patch(rect)
    plt.savefig('tmp.png')
    tmp = Image.open("tmp.png")
    return tmp

def search(results, image_root):
    colors = ['red', 'green', 'yellow']
    # print("ap:{}, acc:{}".format(results['ap'], results['acc']))
    search_result = []
    img = visualize(image_root, results['probe_img'], results['probe_roi'], colors[2]) # query
    search_result.append(img)
    for j in range(0, 5):
        info = results['gallery'][j]
        img = visualize(image_root, info['img'], info['roi'][:4], colors[info['correct']])  # gallery
        search_result.append(img)
    # cat_image(search_result)
    return True


def search_dataset(evaluate):
    results = evaluate['results']
    image_root = evaluate['image_root']
    for idx in range(0, len(results)):
        search(results[idx], image_root)
        break

search_dataset(cuhk_evaluate1)   # ['cuhk_evaluate1', 'prw_evaluate3']
